# **Emotion Detection - FER2013**

## **Downloading Data from Kagglehub**

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Zain Faisal\.cache\kagglehub\datasets\msambare\fer2013\versions\1


## **Importing Necessary Libararies**

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## **Data Paths**

In [ ]:
# Define paths
data_dir = r"C:\Users\Zain Faisal\.cache\kagglehub\datasets\msambare\fer2013\versions\1"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

## **Data Augmentation**

In [7]:
# Image Data Generator with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=15,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

## **Loading Training and Testing data**

In [ ]:
# Load Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## **CNN Model**

In [9]:
# Deep CNN Model
cnn_model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## **Using Pretrained Model**

In [10]:
# Pre-trained Model (EfficientNetB0)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(7, activation='softmax')(x)

efficient_model = Model(inputs=base_model.input, outputs=output_layer)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


## **Compilation of Model**

In [11]:
# Compile Models
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
efficient_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## **Model Checkpoints**

In [12]:
# Model Checkpoints
checkpoint_cnn = ModelCheckpoint("best_cnn_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')
checkpoint_efficient = ModelCheckpoint("best_efficient_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

## **Model Training**

In [15]:
# Train Models
history_cnn = cnn_model.fit(train_generator, validation_data=test_generator, epochs=20, callbacks=[checkpoint_cnn, early_stopping])
history_efficient = efficient_model.fit(train_generator, validation_data=test_generator, epochs=20, callbacks=[checkpoint_efficient, early_stopping])

Epoch 1/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 33s 73ms/step - accuracy: 0.6166 - loss: 1.0272 - val_accuracy: 0.5903 - val_loss: 1.1403
Epoch 2/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 37s 64ms/step - accuracy: 0.6248 - loss: 1.0089 - val_accuracy: 0.5770 - val_loss: 1.1437
Epoch 3/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6273 - loss: 1.0011

449/449 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.6273 - loss: 1.0012 - val_accuracy: 0.6113 - val_loss: 1.0451
Epoch 4/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.6261 - loss: 0.9942 - val_accuracy: 0.6009 - val_loss: 1.0734
Epoch 5/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 29s 65ms/step - accuracy: 0.6276 - loss: 0.9923 - val_accuracy: 0.5685 - val_loss: 1.1639
Epoch 6/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 28s 62ms/step - accuracy: 0.6353 - loss: 0.9865 - val_accuracy: 0.6053 - val_loss: 1.0753
Epoch 7/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.6415 - loss: 0.9581 - val_accuracy: 0.6080 - val_loss: 1.0798
Epoch 8/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 29s 65ms/step - accuracy: 0.6418 - loss: 0.9632 - val_accuracy: 0.5818 - val_loss: 1.1511
Epoch 1/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.2497 - loss: 1.8095 - val_accuracy: 0.2471 - val_loss: 1.8132
Epoch 2/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 28s 63ms/step - accuracy: 0.2491 - loss: 1.8123 - val_accurac

## **Evaluation of Model and selecting best one**

In [16]:
# Evaluate and Select Best Model
cnn_val_acc = max(history_cnn.history['val_accuracy'])
efficient_val_acc = max(history_efficient.history['val_accuracy'])

if efficient_val_acc > cnn_val_acc:
    best_model = efficient_model
    best_model.save("best_model.h5")
    print("EfficientNetB0 model selected as best.")
else:
    best_model = cnn_model
    best_model.save("best_model.h5")
    print("Deep CNN model selected as best.")

Deep CNN model selected as best.


## **Real Time Detection**

In [17]:
# Real-time Emotion Detection via Webcam
def real_time_emotion_detection():
    model = tf.keras.models.load_model("best_model.h5")
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (48, 48)) / 255.0
            face = np.expand_dims(face, axis=0)
            face = np.expand_dims(face, axis=-1)

            prediction = model.predict(face)
            emotion = list(train_generator.class_indices.keys())[np.argmax(prediction)]

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run Emotion Detection (Uncomment to test in real-time)
# real_time_emotion_detection()